In [ ]:
import numpy as np
import pandas as pd
from database_tools.io.wfdb import generate_record_paths, get_data_record, get_header_record, header_has_signals, get_signal
from database_tools.datastores.signals import SignalStore, SignalGroup
from database_tools.datastores.records import WaveformRecord
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'
# pd.options.display.max_rows = 100
# pd.options.display.max_columns = 100

In [1]:
from database_tools.tools.dataset import ConfigMapper

cm = ConfigMapper('/home/cam/Documents/database_tools/data/mimic3-data-20230407/config.ini')

In [ ]:
import json

x = json.dumps(dict(ppg=[0, 1, 2], abp=[3, 4, 5])) + '\n'
x += json.dumps(dict(ppg=[6, 7, 8], abp=[9, 10, 11])) + '\n'
'\n' in x

In [ ]:
import wfdb
from alive_progress import alive_bar

signals = ['PLETH', 'ABP']
min_length = 75000  # 10 min in samples (125 Hz)
n_segments = 50

valid_segs = []
with alive_bar(total=n_segments, bar='brackets', force_tty=True) as bar:
    for path in ['31/3162326']:

        # get patient layout header
        layout = get_header_record(path=path, record_type='layout')
        if layout is None: continue  # fx returns None if file DNE

        # check if header has provided signals
        if not header_has_signals(layout, signals): continue

        # get patient master header
        master = get_header_record(path=path, record_type='data')
        if master is None: continue

        # zip segment names and lengths
        for seg_name, n_samples in zip(master.seg_name, master.seg_len):
        
            # check segment length
            if (n_samples > min_length) & (seg_name != '~'):  # '~' indicates data is missing
                seg_path = path + '/' + seg_name

                # Get segment header
                hea = get_header_record(path=seg_path, record_type='data')
                if hea is None: continue

                # Check if segment has provided signals and append
                if header_has_signals(hea, signals):
                    valid_segs.append(seg_path)
                    if len(valid_segs) > n_segments:
                        break
                    bar()  # iterate loading bar

In [ ]:
from tqdm import tqdm

ppg, abp = [], []
for seg in tqdm(valid_segs[0:5]):
    rec = get_data_record(path=seg, record_type='waveforms')
    p = get_signal(rec, sig='PLETH')
    a = get_signal(rec, sig='ABP')
    ppg.append(p)
    abp.append(a)
ppg = np.array(ppg)
abp = np.array(abp)

In [ ]:
fig = go.Figure()

fig.add_scatter(y=ppg[0])

In [ ]:
from neurokit2.ppg.ppg_findpeaks import _ppg_findpeaks_bishop

def find_peaks(sig, show=False, **kwargs):
   """Modified version of neuroki2 ppg_findpeaks method. Returns peaks and troughs
      instead of just peaks. See neurokit2 documentation for original function.
   """
   peaks, troughs = _ppg_findpeaks_bishop(sig, show=show, **kwargs)
   return dict(peaks=peaks[0], troughs=troughs[0])

def detect_notches(sig: np.ndarray, peaks: np.ndarray, troughs: np.ndarray, dx: int = 10) -> list:
   """Detect dichrotic notch by find the maximum velocity
      at least 10 samples after peak and 30 samples before
      the subsequent trough.

   Args:
         sig (np.ndarray): Cardiac signal.
         peaks (list): List of signal peak indices.
         troughs (list): List of signal trough indices.
         dx (int, optional): Spacing between sig values (for np.gradient). Defaults to 10.

   Returns:
         notches (list): List of dichrotic notch indices.
   """
   # always start with first peak
   try:
       if peaks[0] > troughs[0]:
           troughs = troughs[1::]
   except IndexError:
       return []

   notches = []
   for i, j in zip(peaks, troughs):
       try:
           vel = np.gradient(sig[i:j], dx)
           vel_len = vel.shape[0]
           n = np.argmax(vel[int(vel_len / 100 * 25):int(vel_len / 100 * 75)])
           notches.append(n + i)  # add first index of slice to get correct notch index
       except ValueError:  # gradient fails if slice of sig is too small
           continue

   # look for a notch after the last peak if the highest index is a peak.
   try:
       if peaks[-1] > troughs[-1]:
           try:
               vel = np.gradient(sig[peaks[-1]::], dx)
               vel_len = vel.shape[0]
               n = np.argmax(vel[int(vel_len / 100 * 25):int(vel_len / 100 * 75)])
               notches.append(n + peaks[-1])
           except ValueError:
               pass
   except IndexError:
       pass

   # remove notches that are just peaks
   notches = np.array(notches)[notches - ]
   return notches

In [ ]:
notches = np.array([125, 300, 390, 450, 500])
peaks = np.array([120, 290, 315, 400, 510])
a, b = np.meshgrid(notches, peaks)
y = b - a
y

In [ ]:
notches[np.array(np.where(np.abs(y) <= 10))[1, :]]

In [ ]:
np.abs(y)

In [ ]:
i = (0, 232000, 234000)

x = ppg[i[0]][i[1]:i[2]]
peaks, troughs = find_peaks(x).values()
notches = detect_notches(x, peaks, troughs)

fig = go.Figure()
fig.add_scatter(y=x)
fig.add_scatter(x=peaks, y=x[peaks], mode='markers')
fig.add_scatter(x=troughs, y=x[troughs], mode='markers')
fig.add_scatter(x=notches, y=x[notches], mode='markers')

In [ ]:
fig = go.Figure()

fig.add_scatter(y=abp[0])